In [ ]:
import json
from cassandra.cluster import Cluster
from collections import defaultdict

cluster_ip="CAMBIAR"

## Lectura hall of fame

In [ ]:
def get_hall_of_fame(country: str):
    # Conectar a Cassandra
    cluster = Cluster([cluster_ip])
    session = cluster.connect('videojuego') 

    # Establecer consistencia más fuerte
    # podríamos usar locar ya que en la práctica todo se ubica en el mismo datacenter, pero para facilitar la escalabilidad dejamos Quorum 
    session.default_consistency_level = "QUORUM"

    # Query 
    query = """
        SELECT Mazmorra_id, Nombre_mazmorra, Email, Nombre_usuario, Tiempo, Fecha 
        FROM Hall_of_fame 
        WHERE Pais = %s;
    """
    
    rows = session.execute(query, (country,))
    
    # Organizar datos en un diccionario agrupado por Mazmorra_id
    dungeon_dict = defaultdict(lambda: {"dungeon_id": 0, "dungeon_name": "", "Top_5": []})

    for row in rows:
        dungeon_id = row.Mazmorra_id
        dungeon_dict[dungeon_id]["dungeon_id"] = dungeon_id
        dungeon_dict[dungeon_id]["dungeon_name"] = row.Nombre_mazmorra
        dungeon_dict[dungeon_id]["Top_5"].append({
            "email": row.Email,
            "user_name": row.Nombre_usuario,
            "time_minutes": row.Tiempo,
            "date": row.Fecha
        })
    
    result = json.dumps(list(dungeon_dict.values()), indent=4)  
    return result


In [ ]:
# Ejemplo de uso
country = "España"
top_players = get_hall_of_fame(country)
print(top_players)


## Lectura Statistics

In [ ]:
def get_user_statistics(user_email: str, dungeon_id: int):
    # Conectar a Cassandra
    cluster = Cluster([cluster_ip]) 
    session = cluster.connect('videojuego') 

    # Establecer consistencia más fuerte
    session.default_consistency_level = "QUORUM"
    
    # Query 
    query = """
        SELECT Tiempo, Fecha 
        FROM Statistic 
        WHERE Email = %s AND Mazmorra_id = %s;
    """
    
    rows = session.execute(query, (user_email, dungeon_id))
    
    # converitr a lista de diccionarios
    result = [{"time_minute": row.Tiempo, "date": row.Fecha} for row in rows]
    
    # Convertir a JSON
    return json.dumps(result, indent=4)


In [ ]:
# Ejemplo de uso
user_email = "user1@mail.com"
dungeon_id = 1
statistics_json = get_user_statistics(user_email, dungeon_id)
print(statistics_json)


## Lectura Top horde

In [ ]:
def get_top_horde(country: str, event_id: int, k: int):
    # Conectar a Cassandra
    cluster = Cluster([cluster_ip])  
    session = cluster.connect('videojuego')  

    # Reducimos consistencia para asegurar velocidad en las lecturas
    session.default_consistency_level = 1

    # Query optimizada
    query = """
        SELECT Email, Nombre_usuario, N_killed 
        FROM Top_horde 
        WHERE Evento_id = %s AND Pais = %s 
        LIMIT %s;
    """
    
    rows = session.execute(query, (event_id, country, k))
    
    # Convertir a lista de diccionarios
    result = [
        {
            "user_id": idx + 1,  # Asignamos un ranking (opcional)
            "user_name": row.Nombre_usuario,
            "email": row.Email,
            "n_killed": row.N_killed
        }
        for idx, row in enumerate(rows)
    ]

    # Convertir a JSON
    return json.dumps(result, indent=4)


In [ ]:
# Ejemplo de uso
country = "España"
event_id = 1
k = 3
top_horde_json = get_top_horde(country, event_id, k)
print(top_horde_json)
